In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import glob
from keras.models import Sequential
from keras.layers import Dense,Conv1D,Conv2D,MaxPooling2D,Flatten,AveragePooling2D,Input,BatchNormalization,AveragePooling1D
from keras.optimizers import RMSprop

from utils.dataloader import read_edf_to_raw, calculate_neg_pos

2022-07-06 12:21:55.119325: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guisoares/catkin_ws/devel/lib:/opt/ros/noetic/lib:/usr/lib/x86_64-linux-gnu/:/home/guisoares/catkin_ws/install/lib/mavlink_sitl_gazebo/plugins
2022-07-06 12:21:55.119353: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [24]:
def create_model_article():
    # Model creation
    model = Sequential()
    # model.add(Input(shape=(23,25600,1)))

    model.add(Conv1D(4, 5, activation='relu', input_shape=(12800, 23), name='conv1'))
    model.add(BatchNormalization(name='batch1'))
    model.add(AveragePooling1D(pool_size=2, strides=2, name='pool1'))
    # output ~(12800,4)

    model.add(Conv1D(4, 6, activation='relu', name='conv2'))
    model.add(BatchNormalization(name='batch2'))
    model.add(AveragePooling1D(pool_size=4, strides=1,name='pool2'))
    # output ~(12800,4)

    model.add(Conv1D(10, 4, activation='relu', name='conv3'))
    model.add(BatchNormalization(name='batch3'))
    model.add(AveragePooling1D(pool_size=2, strides=2, name='pool3'))
    # output ~(6400,4)

    model.add(Conv1D(10, 4, activation='relu', name='conv4'))
    model.add(BatchNormalization(name='batch4'))
    model.add(AveragePooling1D(pool_size=2, strides=4, name='pool4'))
    # output ~(1200,4)

    model.add(Conv1D(15, 2, activation='relu', name='conv5'))
    model.add(BatchNormalization(name='batch5'))
    model.add(AveragePooling1D(pool_size=2, strides=4, name='pool5'))

    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [23]:
def create_model_customized():
    # Model creation
    model = Sequential()
    # model.add(Input(shape=(23,25600,1)))

    model.add(Conv2D(16, 3, activation='relu', padding='same', input_shape=(25600, 23, 1), name='conv1'))
    model.add(BatchNormalization(name='batch1'))
    model.add(MaxPooling2D(pool_size=(32,2), name='pool1'))

    model.add(Conv2D(8, 5, activation='relu', padding='same', name='conv2'))
    model.add(BatchNormalization(name='batch2'))
    model.add(MaxPooling2D(pool_size=(16,2), name='pool2'))

    model.add(Conv2D(4, 7, activation='relu', padding='same', name='conv3'))
    model.add(BatchNormalization(name='batch3'))
    model.add(MaxPooling2D(pool_size=(8,2), name='pool3'))

    model.add(Conv2D(2, 3, activation='relu',  padding='same', name='conv7'))
    model.add(BatchNormalization(name='batch7'))
    model.add(MaxPooling2D(pool_size=(4,2), name='pool4'))

    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [27]:
model = create_model_article()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv1D)              (None, 12796, 4)          464       
                                                                 
 batch1 (BatchNormalization)  (None, 12796, 4)         16        
                                                                 
 pool1 (AveragePooling1D)    (None, 6398, 4)           0         
                                                                 
 conv2 (Conv1D)              (None, 6393, 4)           100       
                                                                 
 batch2 (BatchNormalization)  (None, 6393, 4)          16        
                                                                 
 pool2 (AveragePooling1D)    (None, 6390, 4)           0         
                                                                 
 conv3 (Conv1D)              (None, 6387, 10)         

In [28]:
opt = keras.optimizers.SGD(learning_rate=0.1)
# Create the loss function object using the wrapper function above
# spec_loss = specificity_loss_wrapper()
model.compile(
            loss=tf.keras.losses.BinaryCrossentropy(), 
            optimizer=opt, 
            metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5), tf.keras.metrics.SensitivityAtSpecificity(0.5)])

In [6]:
NEW_DATASET_PATH = "/media/guisoares/guisoares-ext-hdd/Datasets/chb-mit-segments/1"

train_files = glob.glob(os.path.join(NEW_DATASET_PATH,'train',"*.npz"))
test_files = glob.glob(os.path.join(NEW_DATASET_PATH,'test',"*.npz"))

print("Training files:")
print(train_files)
print("Testing files:")
print(test_files)

Training files:
['/media/guisoares/guisoares-ext-hdd/Datasets/chb-mit-segments/1/train/chb01_segments.npz', '/media/guisoares/guisoares-ext-hdd/Datasets/chb-mit-segments/1/train/chb02_segments.npz', '/media/guisoares/guisoares-ext-hdd/Datasets/chb-mit-segments/1/train/chb08_segments.npz']
Testing files:
['/media/guisoares/guisoares-ext-hdd/Datasets/chb-mit-segments/1/test/chb05_segments.npz']


In [7]:
# Load test arrays to memory
x_test = []
y_test = []
for file in test_files:
    test_arrays = np.load(file)
    x_test.append(test_arrays['arr_0'])
    y_test.append(test_arrays['arr_1'])

x_test = np.array(x_test)
x_test = x_test.reshape((x_test.shape[0]*x_test.shape[1], x_test.shape[2], x_test.shape[3]))
x_test = np.swapaxes(x_test, 1, 2)

y_test = np.array(y_test)
y_test = y_test.reshape((y_test.shape[0]*y_test.shape[1], -1))

In [8]:
def calculate_neg_pos(y):
    unique, counts = np.unique(y, return_counts=True)
    dic = dict(zip(unique, counts))
    neg = dic[0]
    pos = dic[1]
    return neg, pos

neg_test, pos_test = calculate_neg_pos(y_test)
print(f"Negatives: {neg_test}, Positives: {pos_test}")

Negatives: 9, Positives: 4


In [29]:
# callback = tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', min_delta=0.05)

epochs = 50
for epoch in range(epochs):
    for file in train_files:

        # y = np.load('signals/chb01/labels.npy')
        # x = np.load('signals/chb01/signals.npy') 

        arrays = np.load(file)
        x_train = arrays['arr_0']
        y_train = arrays['arr_1']
        x_train = np.swapaxes(x_train, 1, 2) #swap axis: N,23,25600 -> N,25600, 23
        # x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, shuffle=True)

        print(f"Epoch: {epoch+1} ", end = "")
        unique, counts = np.unique(y_train, return_counts=True)
        print(f"Train :{dict(zip(unique, counts))} ", end="")

        history = model.fit(x_train, y_train, batch_size=1, epochs=1, validation_data=(x_test,y_test), verbose=2)

        if history.history['binary_accuracy'][0] > 0.99:
            break
    if history.history['binary_accuracy'][0] > 0.99:
        break
    
        # train_metrics = model.train_on_batch(x_train, y_train, return_dict=True)
        # test_metrics = model.test_on_batch(x_test, y_test, return_dict=True)

        # print(train_metrics, test_metrics)



Epoch: 1 Train :{0: 7, 1: 3} 10/10 - 4s - loss: 0.6546 - binary_accuracy: 0.5000 - sensitivity_at_specificity_3: 0.3333 - val_loss: 16.1904 - val_binary_accuracy: 0.6923 - val_sensitivity_at_specificity_3: 0.0000e+00 - 4s/epoch - 355ms/step
Epoch: 1 Train :{0: 7, 1: 3} 10/10 - 0s - loss: 0.9146 - binary_accuracy: 0.7000 - sensitivity_at_specificity_3: 0.0000e+00 - val_loss: 4.9822 - val_binary_accuracy: 0.6923 - val_sensitivity_at_specificity_3: 0.0000e+00 - 342ms/epoch - 34ms/step
Epoch: 1 Train :{0: 14, 1: 6} 20/20 - 0s - loss: 0.6445 - binary_accuracy: 0.7000 - sensitivity_at_specificity_3: 0.1667 - val_loss: 11.3126 - val_binary_accuracy: 0.6923 - val_sensitivity_at_specificity_3: 0.0000e+00 - 460ms/epoch - 23ms/step
Epoch: 2 Train :{0: 7, 1: 3} 10/10 - 0s - loss: 0.6818 - binary_accuracy: 0.7000 - sensitivity_at_specificity_3: 0.0000e+00 - val_loss: 6.5488 - val_binary_accuracy: 0.6923 - val_sensitivity_at_specificity_3: 0.0000e+00 - 300ms/epoch - 30ms/step
Epoch: 2 Train :{0: 7, 

KeyboardInterrupt: 

In [14]:
history.history.keys

[1.0]

In [20]:
y_pred = model.predict(x_train)
print(y_pred)
y_pred = np.where(y_pred < 0.5, 0, 1)
unique, counts = np.unique(y_pred, return_counts=True)
print(dict(zip(unique, counts)))

unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

1/1 [==============================] - 0s 98ms/step
[[0.00143324]
 [0.00143306]
 [0.0014334 ]
 [0.00143323]
 [0.00143336]
 [0.00143315]
 [0.00143329]
 [0.00143332]
 [0.00143399]
 [0.00143367]]
{0: 10}
{0: 7, 1: 3}


In [12]:
import keras.backend as K
def compute_binary_specificity(y_pred, y_true):
    """Compute the confusion matrix for a set of predictions.

    Parameters
    ----------
    y_pred   : predicted values for a batch if samples (must be binary: 0 or 1)
    y_true   : correct values for the set of samples used (must be binary: 0 or 1)

    Returns
    -------
    out : the specificity
    """

    # check_binary(K.eval(y_true))    # must check that input values are 0 or 1
    # check_binary(K.eval(y_pred))    # 

    TN = np.logical_and(K.eval(y_true) == 0, K.eval(y_pred) == 0)
    FP = np.logical_and(K.eval(y_true) == 0, K.eval(y_pred) == 1)

    # as Keras Tensors
    TN = K.sum(K.variable(TN))
    FP = K.sum(K.variable(FP))

    specificity = TN / (TN + FP + K.epsilon())
    return specificity

def specificity_loss_wrapper():
    """A wrapper to create and return a function which computes the specificity loss, as (1 - specificity)

    """
    # Define the function for your loss
    def specificity_loss(y_true, y_pred):
        return 1.0 - compute_binary_specificity(y_true, y_pred)

    return specificity_loss    # we return this function object